<a href="https://colab.research.google.com/github/mini1115/TIL/blob/master/%EB%B9%85%EB%B6%84%EA%B8%B0%20%EA%B8%B0%EC%B6%9C/7%ED%9A%8C%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [259]:
from google.colab import files
uploaded = files.upload()

Saving system_cpu.csv to system_cpu.csv
Saving clam.csv to clam.csv


# **작업형1**

Q1.
- 결측치가 있는 행을 제거한 후
- 학생이 가장 많이 수강한 과목을 찾고
- 해당 과목 점수를 표준화(스탠다드 스케일)한 후 가장 큰 표준화 값을 구하시오.

In [195]:
import pandas as pd
df = pd.read_csv('student_assessment.csv')

df =df.dropna()
id = df['id_assessment'].value_counts().idxmax()
cond = df['id_assessment']==id
df =df[cond]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df['score'] = scaler.fit_transform(df[['score']])
round(df['score'].max(),3)

2.183

Q2. DE ~ DE77 컬럼중 주가지수의 종가'CLOSE' 와 가장 상관관계가 높은 변수를 찾아 해당 변수의 평균값을 구하시오.


In [207]:
import pandas as pd
df = pd.read_csv('stock_market.csv')
df_corr = df.corr()['close'].abs()

col = df_corr.loc['DE1':'DE77'].idxmax()

round(df['DE14'].mean(),4)

'DE14'

Q3. IQR을 이용해 이산화탄소(CO2)이상치 수를 찾으시오.
- Q1 : 0.25
- Q3 : 0.75
- IQR : Q3-Q1
- 하한선 : Q1 - 1.5 * IQR
- 상한선 : Q3 + 1.5 * IQR

In [218]:
import pandas as pd
df = pd.read_csv('air_quality.csv')

Q1 = df['CO2'].quantile(0.25)
Q3 = df['CO2'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - (1.5*IQR)
upper = Q3 + (1.5*IQR)

cond1 = df['CO2'] < lower
cond2 = df['CO2'] > upper
ans = df[cond1 | cond2]
len(ans)

304

# **작업형2**

Q. W-mart 매장의 판매 데이터를 기반으로 판매액을 예측하시오.
  - 예측할 컬럼 total(총 판매액)
  - pred : 예측된 총 판매액
  - 제출 파일명 'result.csv'
  - 평가지표 RMSE
  

In [258]:
train = pd.read_csv('mart_train.csv')
test = pd.read_csv('mart_test.csv')

# train.shape,test.shape
# train.head()
# train.isnull().sum()
# test.isnull().sum()

# train.describe(include='O')
target = train.pop('total')

# combined = pd.concat([train,test])
# combined_oh = pd.get_dummies(combined)
# train = combined_oh[:len(train)]
# test = combined_oh[:len(train)]

from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O')
for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.fit_transform(test[col])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = train.select_dtypes(include=['int64', 'float64']).columns
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)
# X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(random_state=0)
# rf.fit(X_tr,y_tr)
# pred = rf.predict(X_val)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)


# import lightgbm as lgb
# lg = lgb.LGBMRegressor(random_state=0,verbose=-1)
# lg.fit(X_tr,y_tr)
# pred = lg.predict(X_val)

from sklearn.metrics import root_mean_squared_error
score = root_mean_squared_error(y_val,pred)
print(score)
#371070.1344734549

pred = lr.predict(test)
sumbit = pd.DataFrame({'pred':pred})
sumbit.to_csv('result.csv',index=False)


371070.1344734549


# **작업형3**

Q.1 로지스틱 회귀
조개 정보를 나타낸 데이터다. 주어진 데이터에서 300개 중 앞에서부터 210개는 TRAIN 뒤에 90개는 TEST로 나눈다.<br/>
모델을 학습할땐 TRAIN데이터를 사용하고 예측할떄는 TEST 데이터 사용한다.<br/>
다중 로지스틱 회귀를 써서 성별을 예측하되 페널티는 부과하지 않는다.


In [283]:
data = pd.read_csv('clam.csv')
train = data[:210]
test = data[210:]
# train.shape,test.shape

import numpy as np
from statsmodels.formula.api import logit
formula = "gender ~ weight"
model = logit(formula,data=train).fit()
odds_ratio = np.exp(model.params['weight'])
round(odds_ratio,4)

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4


1.0047

In [287]:
data = pd.read_csv('clam.csv')
train = data[:210]
test = data[210:]

train.columns
from statsmodels.formula.api import logit

formula = 'gender ~ age+length+diameter+height+weight'
model = logit(formula,data=train).fit()
model.summary()
round(model.llf * -2,2)

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4


286.93

In [297]:
data = pd.read_csv('clam.csv')
train = data[:210]
test = data[210:]

from statsmodels.formula.api import logit
formula = "gender ~ weight"
model = logit(formula,data=test).fit()
target = test.pop('gender')

from sklearn.metrics import accuracy_score
pred = model.predict(test)
# 각 관측치가 1에 속할 확률
pred = (pred>0.5).astype(int)

accuracy_score = accuracy_score(target,pred)
ans = (1-accuracy_score)
print(round(ans,3))

Optimization terminated successfully.
         Current function value: 0.688801
         Iterations 4
0.422


- Q1. ERP와 상관관계가 높은 값을 구하시오.<BR/>
- Q2. CPU 컬럼이 100미만인 것만 찾아 ERP종속변수, 나머지 변수들은 독립변수로 설정해 <BR/>
선형 회귀 모델을 만들고 적합한 결정계수를 구하시오.
- Q3. Q2의 독립변수중 가장 높은 변수를 찾으시오

In [310]:
df = pd.read_csv('system_cpu.csv')
data = df.corr()
data['ERP'].sort_values(ascending =False)
# Feature3	0.882194
df = df.loc[df['CPU']<100]

from statsmodels.formula.api import ols
formula = 'ERP ~	Feature1 +	Feature2	+ Feature3 + CPU'
model = ols(formula,data=df).fit()
model.summary()
model.pvalues

,0
Intercept,9.630266e-03
Feature1,6.841646e-01
Feature2,5.717204e-01
Feature3,4.300038e-17
CPU,5.255548e-02
